BLIP, InstructBLIP, CLIP

In [1]:
# ===============================
# 1. 환경 설정
# ===============================
!pip install transformers pillow accelerate torch torchvision --quiet

from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ===============================
# 2. 이미지 불러오기
# ===============================
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
image = Image.open(image_path).convert("RGB")
image.show()

# ===============================
# 3. 모델별 설정 및 캡션 생성
# ===============================

results = {}

# ---- (1) BLIP: Salesforce/blip-image-captioning-large ----
print("🔹 Running BLIP...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

inputs = blip_processor(image, return_tensors="pt").to(device)
out = blip_model.generate(**inputs, max_length=100)
results["BLIP"] = blip_processor.decode(out[0], skip_special_tokens=True)

# ---- (2) InstructBLIP: Salesforce/instructblip-flan-t5-xl ----
print("🔹 Running InstructBLIP...")
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration

ins_processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-flan-t5-xl")
ins_model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-flan-t5-xl").to(device)

prompt = "Describe this image in detail, including objects, colors, positions, sizes, shapes, and atmosphere."
inputs = ins_processor(images=image, text=prompt, return_tensors="pt").to(device)
out = ins_model.generate(**inputs, max_new_tokens=100)
results["InstructBLIP"] = ins_processor.tokenizer.decode(out[0], skip_special_tokens=True)


# ---- (3) CLIP Interrogator (optional, aesthetic caption) ----
try:
    print("🔹 Running CLIP Interrogator...")
    !pip install clip-interrogator --quiet
    from clip_interrogator import Config, Interrogator

    import open_clip
    ci = Interrogator(Config(clip_model_name="ViT-L-14/openai"))
    results["CLIP Interrogator"] = ci.interrogate(image)
except Exception as e:
    results["CLIP Interrogator"] = f"⚠️ CLIP Interrogator not run: {e}"

# ===============================
# 4. 결과 출력
# ===============================
print("\n==============================")
print("📸 Detailed Caption Comparison")
print("==============================")
for model_name, caption in results.items():
    print(f"\n🔸 {model_name}:\n{caption}")


✅ Using device: cuda


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🔹 Running BLIP...


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(
ViT-L-14_openai_artists.safetensors: 100%|██████████| 16.2M/16.2M [00:00<00:00, 105MB/s] 
ViT-L-14_openai_flavors.safetensors: 100%|██████████| 155M/155M [00:00<00:00, 237MB/s]
ViT-L-14_openai_mediums.safetensors: 100%|██████████| 146k/146k [00:00<00:00, 6.00MB/s]
ViT-L-14_openai_movements.safetensors: 100%|██████████| 307k/307k [00:00<00:00, 8.61MB/s]
ViT-L-14_openai_trendings.safetensors: 100%|██████████| 111k/111k [00:00<00:00, 5.89MB/s]
ViT-L-14_openai_negative.safetensors: 100%|██████████| 63.2k/63.2k [00:00<00:00, 4.76MB/s]


Loaded CLIP model and data in 9.26 seconds.


/usr/local/lib/python3.12/dist-packages/clip_interrogator/clip_interrogator.py:200: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
  0%|          | 0/55 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/clip_interrogator/clip_interrogator.py:376: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 55/55 [00:00<00:00, 182.31it/s]
/usr/local/lib/python3.12/dist-packages/clip_interrogator/clip_interrogator.py:271: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
Flavor chain:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/clip_interrogator/clip_interrogator.py:260: FutureWarning: `torc


📸 Detailed Caption Comparison

🔸 BLIP:
drawing of a tree with a bunch of flowers in it

🔸 InstructBLIP:
The image features a drawing of a tree with a circular shape and a few leaves. The tree is drawn in black and white, with a few leaves scattered around the tree. The drawing is placed on a white background, with the tree positioned in the middle of the image. The drawing is surrounded by a few other objects, including a pencil, a ruler, and a ruler. The pencil is placed in the middle of the image, while

🔸 CLIP Interrogator:
drawing of a tree with a bunch of flowers in it, black and white logo, smooth rounded shapes, a child's drawing, loosely cropped, lower quality, artist rendition, seedlings, spiraling upward, single line, artist's impression



/usr/local/lib/python3.12/dist-packages/clip_interrogator/clip_interrogator.py:280: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


In [ ]:
pip install transformers accelerate bitsandbytes pillow torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 37.2 MB/s eta 0:00:00


Kosmos-2

In [2]:
!pip install transformers accelerate pillow torchvision --quiet

from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch

# ===============================
# 1️⃣ 이미지 경로 설정
# ===============================
# 사용자의 이미지 경로에 맞게 이 부분을 수정해야 함
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
try:
    image = Image.open(image_path).convert("RGB")
    print("✅ 이미지 불러오기 완료.")
except FileNotFoundError:
    print(f"❌ 오류: 이미지 경로를 찾을 수 없음. 경로를 확인해주세요: {image_path}")
    exit()

# ===============================
# 2️⃣ Kosmos-2 모델 불러오기
# ===============================
print("⏳ Kosmos-2 모델 로딩 시작...")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")
# float16으로 메모리를 절약하며 로딩함
model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224", torch_dtype=torch.float16)
print("✅ 모델 로딩 완료.")

# ===============================
# 3️⃣ 캡션 생성 함수 (최소화)
# ===============================
def generate_caption_basic(image, processor, model):
    # 빈 프롬프트를 사용하여 기본 캡션 생성을 유도함
    prompt = ""

    # 텍스트와 이미지 입력
    inputs = processor(text=prompt, images=image, return_tensors="pt")

    # 장치 설정
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    for k in inputs:
        inputs[k] = inputs[k].to(device)

    # 생성 (max_new_tokens=50으로 설정해 불필요하게 길어지는 것을 방지)
    outputs = model.generate(**inputs, max_new_tokens=50)

    # 디코딩. 특수 토큰을 제외하고 문장으로 변환
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    # ⚠️ 최소한의 정리만 함: 이미지 관련 특수 토큰만 제거
    caption = caption.replace("<image>", "").replace("</image>", "").strip()

    return caption

# ===============================
# 4️⃣ 실행 및 출력
# ===============================
print("⏳ 캡션 생성 중...")
caption_basic = generate_caption_basic(image, processor, model)

print("\n--- 결과 ---")
print("🔹 Kosmos-2 Caption (기본 출력):")
print(caption_basic)
print("----------")

✅ 이미지 불러오기 완료.
⏳ Kosmos-2 모델 로딩 시작...


preprocessor_config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/6.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

✅ 모델 로딩 완료.
⏳ 캡션 생성 중...

--- 결과 ---
🔹 Kosmos-2 Caption (기본 출력):
. the, to and of as in I that' for is was- on’ it with The as at bet he have from by are " you his “ this said not has an ( but had we her they will my or were their): up about out who one all been she can more would It. The tree is a symbol of life, love, and hope. It is a tree that is full of life and love. It symbolizes the beginning of a new life, the beginning and the end of a relationship, and the beginning or the
----------


In [3]:
!pip install transformers accelerate pillow torchvision --quiet

from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch
import re # 문자열을 정리(정규식)하기 위해 필요함

# ===============================
# 1️⃣ 이미지 경로 설정
# ===============================
# 사용자의 이미지 경로에 맞게 이 부분을 수정해야 함
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
try:
    image = Image.open(image_path).convert("RGB")
    print("✅ 이미지 불러오기 완료.")
except FileNotFoundError:
    print(f"❌ 오류: 이미지 경로를 찾을 수 없음. 경로를 확인해주세요: {image_path}")
    exit()

# ===============================
# 2️⃣ Kosmos-2 모델 불러오기
# ===============================
print("⏳ Kosmos-2 모델 로딩 시작...")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")
# float16으로 메모리를 절약하며 로딩함
model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224", torch_dtype=torch.float16)
print("✅ 모델 로딩 완료.")

# ===============================
# 3️⃣ 캡션 생성 함수 (노이즈 제거 기능 추가)
# ===============================
def generate_caption_clean(image, processor, model):
    # 빈 프롬프트를 사용하여 기본 캡션 생성을 유도함
    prompt = ""

    inputs = processor(text=prompt, images=image, return_tensors="pt")

    # 장치 설정
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    for k in inputs:
        inputs[k] = inputs[k].to(device)

    # 생성 (max_new_tokens=100으로 설정)
    outputs = model.generate(**inputs, max_new_tokens=100)

    caption = processor.decode(outputs[0], skip_special_tokens=True)

    # 🌟🌟🌟 노이즈 및 불필요 문자열 제거 과정 🌟🌟🌟

    # 1. <image>, <grounding> 같은 특수 토큰 제거
    caption = caption.replace("<image>", "").replace("</image>", "").replace("<grounding>", "").strip()

    # 2. <patch_index_xxxx> 형태나 <phrase>, <object> 같은 모든 HTML 태그 형태 제거
    caption = re.sub(r'<[^>]+>', '', caption).strip()

    # 3. 모델이 출력한 불필요한 단어 목록(노이즈)을 제거
    # 이전에 나왔던 패턴('. the, to and of as in I that'...)이 끝나고 문장이 시작되는 지점을 찾음

    # 3-1. 문장부호 또는 공백으로 시작하는 모든 단어/기호를 제거 시도
    # 문장의 시작이 알파벳 대문자로 시작하는 부분(A-Z)을 찾아서 그 앞을 모두 잘라냄
    match = re.search(r'[A-Z]', caption)
    if match:
        # 첫 번째 대문자가 시작하는 위치부터 문장을 다시 시작함
        caption = caption[match.start():].strip()
    else:
        # 대문자가 없다면, 그냥 문장 시작의 특수 기호와 공백만 제거함
        caption = re.sub(r'^\s*[\.,:;!]+\s*', '', caption).strip()

    # 4. 혹시라도 남아있을 수 있는 'the'나 'it' 같은 소문자 시작 단어를 문장 시작에서 한 번 더 정리
    caption = re.sub(r'^(the|to|and|of|as|in|I|that|for|is|was|on|it)\s*', '', caption, flags=re.IGNORECASE).strip()

    return caption

# ===============================
# 4️⃣ 실행 및 출력
# ===============================
print("⏳ 캡션 생성 및 정리 중...")
caption_clean = generate_caption_clean(image, processor, model)

print("\n--- 결과 ---")
print("🔹 Kosmos-2 Caption (노이즈 제거 후):")
print(caption_clean)
print("----------")

✅ 이미지 불러오기 완료.
⏳ Kosmos-2 모델 로딩 시작...
✅ 모델 로딩 완료.
⏳ 캡션 생성 및 정리 중...

--- 결과 ---
🔹 Kosmos-2 Caption (노이즈 제거 후):
that' for is was- on’ it with The as at bet he have from by are " you his “ this said not has an ( but had we her they will my or were their): up about out who one all been she can more would It. The tree is a symbol of life, love, and hope. It is a tree that is full of life and love. It symbolizes the beginning of a new life, the beginning and the end of a relationship, and the beginning or the end. The symbol of the tree is also a symbol for the beginning, the end, and life itself. The beginning of life is represented by the tree, and it symbolizes hope, love and life. The end of life symbolizes death,
----------


In [4]:
!pip install transformers accelerate pillow torchvision --quiet

from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch
import re # 문자열 정리(정규식)를 위해 필요함

# ===============================
# 1️⃣ 이미지 경로 설정
# ===============================
# 사용자의 이미지 경로에 맞게 이 부분을 수정해야 함
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
try:
    image = Image.open(image_path).convert("RGB")
    print("✅ 이미지 불러오기 완료.")
except FileNotFoundError:
    print(f"❌ 오류: 이미지 경로를 찾을 수 없음. 경로를 확인해주세요: {image_path}")
    exit()

# ===============================
# 2️⃣ Kosmos-2 모델 불러오기
# ===============================
print("⏳ Kosmos-2 모델 로딩 시작...")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")
# float16으로 메모리를 절약하며 로딩함
model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224", torch_dtype=torch.float16)
print("✅ 모델 로딩 완료.")

# ===============================
# 3️⃣ 캡션 생성 함수 (프롬프트 사용 및 강화된 노이즈 제거)
# ===============================
def generate_caption_with_prompt_and_clean(image, processor, model):
    # 🌟🌟🌟 명확한 프롬프트 사용 🌟🌟🌟
    # 이미지의 내용을 자세히 묘사하도록 지시하는 프롬프트
    prompt = "<grounding>A detailed description of the image, including all visible objects and their attributes:"

    inputs = processor(text=prompt, images=image, return_tensors="pt")

    # 장치 설정
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    for k in inputs:
        inputs[k] = inputs[k].to(device)

    # 생성 (max_new_tokens를 150으로 늘려 충분한 설명을 얻도록 함)
    outputs = model.generate(**inputs, max_new_tokens=150)

    caption = processor.decode(outputs[0], skip_special_tokens=True)

    # 🌟🌟🌟 강화된 노이즈 및 불필요 문자열 제거 과정 🌟🌟🌟

    # 1. <image>, <grounding> 같은 특수 토큰 제거
    caption = caption.replace("<image>", "").replace("</image>", "").replace("<grounding>", "").strip()

    # 2. <patch_index_xxxx> 형태나 <phrase>, <object> 같은 모든 HTML/XML 태그 형태 제거
    caption = re.sub(r'<[^>]+>', '', caption).strip()

    # 3. 모델이 출력한 불필요한 단어 목록(노이즈)을 제거
    # 문장의 시작이 알파벳 대문자로 시작하는 부분(A-Z)을 찾아서 그 앞을 모두 잘라냄
    # 이는 'the, to and of...' 와 같은 이상한 시작 부분을 제거하는 데 효과적임.
    match = re.search(r'[A-Z]', caption)
    if match:
        caption = caption[match.start():].strip()
    else:
        # 대문자가 없다면, 그냥 문장 시작의 특수 기호와 공백만 제거함
        caption = re.sub(r'^\s*[\.,:;!]+\s*', '', caption).strip()

    # 4. 프롬프트가 캡션에 그대로 반복되어 포함될 경우, 이를 제거
    # prompt 문자열을 다시 검색하여 제거. flags=re.IGNORECASE로 대소문자 무시
    # re.escape는 prompt 내의 특수문자들이 정규식으로 해석되지 않도록 함
    caption = re.sub(re.escape(prompt.replace("<grounding>", "")), '', caption, flags=re.IGNORECASE, count=1).strip()

    # 5. 혹시라도 남아있을 수 있는 'the'나 'it' 같은 소문자 시작 단어를 문장 시작에서 한 번 더 정리
    caption = re.sub(r'^(the|to|and|of|as|in|I|that|for|is|was|on|it)\s*', '', caption, flags=re.IGNORECASE).strip()

    # 6. 문장 끝에 불필요하게 끊긴 부분 정리 (예: ... or the)
    # 일반적인 문장 끝 패턴(마침표, 물음표, 느낌표)이 아닌 경우, 가장 마지막 문장 부호까지만 남김
    last_punc_match = re.search(r'[.?!](?=[^.?!]*$)', caption)
    if last_punc_match:
        caption = caption[:last_punc_match.end()].strip()

    return caption

# ===============================
# 4️⃣ 실행 및 출력
# ===============================
print("⏳ 캡션 생성 및 정리 중...")
caption_final = generate_caption_with_prompt_and_clean(image, processor, model)

print("\n--- 결과 ---")
print("🔹 Kosmos-2 Caption (프롬프트 사용 및 강화된 정리):")
print(caption_final)
print("----------")

✅ 이미지 불러오기 완료.
⏳ Kosmos-2 모델 로딩 시작...
✅ 모델 로딩 완료.
⏳ 캡션 생성 및 정리 중...

--- 결과 ---
🔹 Kosmos-2 Caption (프롬프트 사용 및 강화된 정리):
that' for is was- on’ it with The as at bet he have from by are " you his “ this said not has an ( but had we her they will my or were their): up about out who one all been she can more would It  A tree, a person, and a dog.
----------


gpt-4o

In [1]:
!pip install openai pillow google-colab --quiet

import os
import base64
from io import BytesIO
from PIL import Image
from openai import OpenAI
from google.colab import userdata # 🌟 Colab 보안 비밀(Secrets) 사용을 위해 추가

# ===============================
# 1️⃣ 설정 및 초기화 (키 불러오기 수정)
# ===============================
# 🌟 Colab 보안 비밀에서 'OPENAI_API_KEY'를 불러와서 클라이언트 초기화에 직접 사용
try:
    # 🌟 Colab 보안 비밀에서 키 값을 가져옴
    api_key = userdata.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("Colab 보안 비밀에서 'OPENAI_API_KEY'를 찾을 수 없음.")

    client = OpenAI(api_key=api_key)
    print("✅ OpenAI 클라이언트 초기화 완료 (Colab 보안 비밀 사용).")

except Exception as e:
    # 키가 제대로 설정되지 않았을 경우 오류 메시지 출력
    print("❌ 오류: Colab 보안 비밀 설정 확인 바람.")
    print(f"   자세한 오류: {e}")
    # client를 정의하지 못했더라도 다음 코드가 NameError를 내지 않도록 None으로 설정
    client = None
    # 프로그램 실행 중단
    exit()

# 🌟 이미지 경로 설정
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
MODEL_NAME = "gpt-4o"

# ===============================
# 2️⃣ 이미지를 Base64로 변환하는 함수
# ===============================
def encode_image_to_base64(image_path):
    """로컬 이미지 파일을 Base64 문자열로 변환함."""
    try:
        img = Image.open(image_path).convert("RGB")
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")
    except FileNotFoundError:
        print(f"❌ 오류: 이미지 경로를 찾을 수 없음: {image_path}")
        return None
    except Exception as e:
        print(f"❌ 오류: 이미지 변환 중 문제 발생: {e}")
        return None

# ===============================
# 3️⃣ GPT-4o로 캡션을 생성하는 함수
# ===============================
def generate_caption_with_openai(client, image_base64):
    if not client:
        return "클라이언트 초기화 오류로 캡션 생성 실패."

    if not image_base64:
        return "이미지 인코딩 실패로 캡션 생성 실패."

    print(f"⏳ GPT 모델({MODEL_NAME})에 캡션 요청 중...")

    # 캡션 요청 프롬프트
    caption_prompt = "이 이미지를 자세하고 간결하게 설명해줘. 이미지에 보이는 주요 물체와 장면의 분위기를 포함해."

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": caption_prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_base64}"
                            },
                        },
                    ],
                }
            ],
            max_tokens=300,
        )

        caption = response.choices[0].message.content
        return caption.strip()

    except Exception as e:
        # API 키는 유효하지만, API 호출 자체에 문제가 생겼을 경우 (예: 잔액 부족 등)
        return f"❌ API 호출 오류 발생 (키는 정상일 수 있음): {e}"

# ===============================
# 4️⃣ 실행 및 출력
# ===============================
# 이미지 Base64 인코딩
base64_image = encode_image_to_base64(image_path)

if base64_image and client: # client가 성공적으로 초기화되었는지 다시 확인
    # 캡션 생성
    final_caption = generate_caption_with_openai(client, base64_image)

    print("\n--- 결과 ---")
    print(f"🔹 {MODEL_NAME} Caption:")
    print(final_caption)
    print("----------")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00
✅ OpenAI 클라이언트 초기화 완료 (Colab 보안 비밀 사용).
⏳ GPT 모델(gpt-4o)에 캡션 요청 중...

--- 결과 ---
🔹 gpt-4o Caption:
이미지에는 간단하게 그려진 나무가 있습니다. 나무는 큰 잎사귀로 덮인 둥근 형태의 나무 꼭대기를 가지고 있으며, 그 안에는 여러 개의 둥근 과일과 꽃이 그려져 있습니다. 나무 줄기는 곧고, 여러 개의 나선형 모양이 포함되어 있습니다. 전체적인 분위기는 단순하고 귀여운 느낌을 주며, 아마도 어린이가 그린 그림일 가능성이 있습니다.
----------


GPT-4o + YOLO

In [2]:
!pip install ultralytics --quiet
from ultralytics import YOLO
import os
from PIL import Image

# ===============================
# YOLO 모델 경로 및 이미지 설정
# ===============================
yolo_model_path = "/content/drive/MyDrive/Colab/T_V_T/pt/68_100best.pt"
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"

# ===============================
# YOLO로 객체 탐지 및 정보 추출 함수
# ===============================
def get_yolo_detections(model_path, image_path):
    print("⏳ YOLO 모델 로딩 및 탐지 시작...")
    try:
        # 모델 로드
        model = YOLO(model_path)
        # 이미지 탐지 실행
        results = model(image_path, conf=0.5, iou=0.7, save=False, verbose=False)

        detections = []
        # 결과 정리
        if results and len(results) > 0:
            result = results[0]
            # 검출된 각 객체에 대해 반복
            for box in result.boxes:
                # 클래스 ID와 확률 추출
                class_id = int(box.cls[0].item())
                confidence = float(box.conf[0].item())

                # 클래스 이름 가져오기 (사용자 정의 모델의 클래스 이름이 필요함)
                class_name = model.names.get(class_id, f"class_{class_id}")

                # 결과 저장
                detections.append(f"{class_name} (확률: {confidence:.2f})")

        if not detections:
            return "탐지된 객체 없음."

        # 탐지 결과를 하나의 문자열로 정리
        detection_string = "탐지된 객체 목록: " + ", ".join(detections)
        print("✅ YOLO 탐지 완료.")
        return detection_string

    except Exception as e:
        return f"❌ YOLO 탐지 오류: {e}"

# ===============================
# 실행
# ===============================
yolo_output = get_yolo_detections(yolo_model_path, image_path)
print("\n--- YOLOv8 탐지 결과 ---")
print(yolo_output)
print("----------")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
⏳ YOLO 모델 로딩 및 탐지 시작...
✅ YOLO 탐지 완료.

--- YOLOv8 탐지 결과 ---
탐지된 객체 목록: 나무전체 (확률: 0.96), 수관 (확률: 0.95), 꽃 (확률: 0.91), 꽃 (확률: 0.90), 꽃 (확률: 0.90), 열매 (확률: 0.87), 열매 (확률: 0.87), 열매 (확률: 0.86), 꽃 (확률: 0.86), 열매 (확률: 0.86), 열매 (확률: 0.85), 기둥 (확률: 0.77)
----------


In [3]:
# 1단계 코드 실행 후 이어서 실행
!pip install openai pillow google-colab --quiet

import os
import base64
from io import BytesIO
from PIL import Image
from openai import OpenAI
from google.colab import userdata

# ===============================
# 1️⃣ 설정 및 초기화
# ===============================
try:
    api_key = userdata.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("Colab 보안 비밀에서 'OPENAI_API_KEY'를 찾을 수 없음.")

    client = OpenAI(api_key=api_key)
    # YOLO 모델 경로 및 이미지 설정 (1단계와 동일)
    image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
    MODEL_NAME = "gpt-4o"
    print("✅ OpenAI 클라이언트 초기화 완료.")

except Exception as e:
    print("❌ 오류: 클라이언트 초기화 실패. Colab 보안 비밀 확인 바람.")
    print(f"   자세한 오류: {e}")
    exit()

# ===============================
# 2️⃣ 이미지를 Base64로 변환하는 함수 (1단계와 동일)
# ===============================
def encode_image_to_base64(image_path):
    try:
        img = Image.open(image_path).convert("RGB")
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")
    except Exception:
        return None

# ===============================
# 3️⃣ GPT-4o로 캡션을 생성하는 함수 (YOLO 정보 사용)
# ===============================
def generate_caption_with_yolo_hint(client, image_base64, yolo_detections):

    # 🌟🌟🌟 프롬프트 수정: YOLO 탐지 정보를 포함 🌟🌟🌟
    caption_prompt = (
        "이 이미지를 자세하고 간결하게 설명해줘. 다음 YOLOv8 탐지 결과를 참고해서 이미지 내용을 더 정확하게 묘사해줘. "
        f"\n\n[YOLO 탐지 정보]: {yolo_detections}"
    )

    print(f"⏳ GPT 모델({MODEL_NAME})에 캡션 요청 중 (YOLO 정보 포함)...")

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": caption_prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_base64}"
                            },
                        },
                    ],
                }
            ],
            max_tokens=300,
        )

        caption = response.choices[0].message.content
        return caption.strip()

    except Exception as e:
        return f"❌ API 호출 오류 발생: {e}"

# ===============================
# 4️⃣ 최종 실행 및 출력
# ===============================
# YOLO 탐지 결과는 이미 1단계에서 yolo_output 변수에 저장됨
base64_image = encode_image_to_base64(image_path)

if base64_image:
    final_caption = generate_caption_with_yolo_hint(client, base64_image, yolo_output)

    print("\n--- 최종 결과 (YOLO 참조) ---")
    print(f"🔹 {MODEL_NAME} Caption:")
    print(final_caption)
    print("----------")

✅ OpenAI 클라이언트 초기화 완료.
⏳ GPT 모델(gpt-4o)에 캡션 요청 중 (YOLO 정보 포함)...

--- 최종 결과 (YOLO 참조) ---
🔹 gpt-4o Caption:
이 이미지는 단순한 그림으로, 큰 나무가 중앙에 그려져 있습니다. 나무는 두껍고 길쭉한 기둥을 가지고 있으며, 기둥에는 원형 무늬가 반복적으로 나타나 있습니다. 수관은 둥글게 그려져 있으며, 그 위로 여러 개의 꽃과 열매가 흩어져 있습니다. 꽃은 여러 군데에 걸쳐 그려져 있으며, 둥근 모양으로 표현되어 있습니다. 열매 또한 곳곳에 매달려 있는 형태로 나타나 있습니다. 전체적으로 이 그림은 애니메이션 스타일의 간단한 나무 스케치를 보여줍니다.
----------


InstructBLIP + YOLOv8

In [4]:
!pip install transformers accelerate torch torchvision ultralytics pillow --quiet

from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from PIL import Image
import torch
from ultralytics import YOLO
import os

# ===============================
# 1. 환경 및 경로 설정
# ===============================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# 🌟 이미지 경로 설정 (수정 필요할 수 있음)
image_path = "/content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG"
# 🌟 YOLOv8 모델 경로 설정 (수정 필요함)
yolo_model_path = "/content/drive/MyDrive/Colab/T_V_T/pt/68_100best.pt"


# ===============================
# 2. 이미지 불러오기 및 YOLO 탐지 함수
# ===============================
try:
    image = Image.open(image_path).convert("RGB")
    print(f"✅ 이미지 불러오기 완료: {image_path}")
except FileNotFoundError:
    print(f"❌ 오류: 이미지 경로를 찾을 수 없음: {image_path}")
    exit()

def get_yolo_detections(model_path, image_path):
    """YOLOv8 모델로 객체를 탐지하고 결과를 텍스트로 정리함."""
    print("⏳ YOLO 모델 로딩 및 탐지 시작...")
    try:
        model = YOLO(model_path)
        # 이미지 탐지 실행 (확신도 0.5 이상, IOU 0.7 이상)
        results = model(image_path, conf=0.5, iou=0.7, save=False, verbose=False)

        detections = []
        if results and len(results) > 0:
            result = results[0]
            for box in result.boxes:
                class_id = int(box.cls[0].item())
                confidence = float(box.conf[0].item())
                class_name = model.names.get(class_id, f"class_{class_id}")
                detections.append(f"{class_name} (확률: {confidence:.2f})")

        if not detections:
            return "탐지된 객체 없음."

        detection_string = "YOLO 탐지 객체: " + ", ".join(detections)
        return detection_string

    except Exception as e:
        return f"❌ YOLO 탐지 오류: {e}"

# YOLO 탐지 실행
yolo_output = get_yolo_detections(yolo_model_path, image_path)
print(f"\n--- YOLO 탐지 결과 ---\n{yolo_output}\n------------------------")


# ===============================
# 3. InstructBLIP 설정 및 캡션 생성
# ===============================
print("🔹 Running InstructBLIP with YOLO Hint...")

ins_processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-flan-t5-xl")
# 메모리 절약을 위해 float16을 사용하여 모델을 로드함
ins_model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-flan-t5-xl", torch_dtype=torch.float16).to(device)


# 🌟🌟🌟 InstructBLIP 프롬프트에 YOLO 결과 추가 🌟🌟🌟
base_prompt = "Describe this image in detail, including objects, colors, positions, sizes, shapes, and atmosphere."
yolo_hint = f"참고 정보 (YOLO 탐지 결과): {yolo_output}. 이 정보를 바탕으로 이미지 설명을 더 정확하게 작성해줘."
final_prompt = f"{base_prompt} {yolo_hint}"

print(f"📝 최종 프롬프트: {final_prompt}")

# 모델 입력 준비
inputs = ins_processor(images=image, text=final_prompt, return_tensors="pt").to(device)

# 캡션 생성
# max_new_tokens를 충분히 주어 자세한 설명을 얻도록 함
out = ins_model.generate(**inputs, max_new_tokens=150)
caption = ins_processor.tokenizer.decode(out[0], skip_special_tokens=True)


# ===============================
# 4. 결과 출력
# ===============================
print("\n==============================")
print("📸 InstructBLIP + YOLO 캡션")
print("==============================")
print(f"🔸 InstructBLIP:\n{caption}")

✅ Using device: cuda
✅ 이미지 불러오기 완료: /content/drive/MyDrive/Colab/T_V_T/htp/test_나무.JPG
⏳ YOLO 모델 로딩 및 탐지 시작...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



--- YOLO 탐지 결과 ---
YOLO 탐지 객체: 나무전체 (확률: 0.96), 수관 (확률: 0.95), 꽃 (확률: 0.91), 꽃 (확률: 0.90), 꽃 (확률: 0.90), 열매 (확률: 0.87), 열매 (확률: 0.87), 열매 (확률: 0.86), 꽃 (확률: 0.86), 열매 (확률: 0.86), 열매 (확률: 0.85), 기둥 (확률: 0.77)
------------------------
🔹 Running InstructBLIP with YOLO Hint...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

📝 최종 프롬프트: Describe this image in detail, including objects, colors, positions, sizes, shapes, and atmosphere. 참고 정보 (YOLO 탐지 결과): YOLO 탐지 객체: 나무전체 (확률: 0.96), 수관 (확률: 0.95), 꽃 (확률: 0.91), 꽃 (확률: 0.90), 꽃 (확률: 0.90), 열매 (확률: 0.87), 열매 (확률: 0.87), 열매 (확률: 0.86), 꽃 (확률: 0.86), 열매 (확률: 0.86), 열매 (확률: 0.85), 기둥 (확률: 0.77). 이 정보를 바탕으로 이미지 설명을 더 정확하게 작성해줘.

📸 InstructBLIP + YOLO 캡션
🔸 InstructBLIP:
a drawing of a tree with a yolo logo
